In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bipolar_disorder"
cohort = "GSE120342"

# Input paths
in_trait_dir = "../../input/GEO/Bipolar_disorder"
in_cohort_dir = "../../input/GEO/Bipolar_disorder/GSE120342"

# Output paths
out_data_file = "../../output/preprocess/Bipolar_disorder/GSE120342.csv"
out_gene_data_file = "../../output/preprocess/Bipolar_disorder/gene_data/GSE120342.csv"
out_clinical_data_file = "../../output/preprocess/Bipolar_disorder/clinical_data/GSE120342.csv"
json_path = "../../output/preprocess/Bipolar_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Aberrant transcriptomes and DNA methylomes define pathways that drive pathogenesis and loss of brain laterality/asymmetry in schizophrenia and bipolar disorder"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['disease state: control', 'disease state: SCZ', 'disease state: BD(-)', 'disease state: BD(+)'], 1: ['laterality: left', 'laterality: right']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
import numpy as np
from typing import Optional, Callable, Dict, Any

# Sample characteristics dictionary from the previous output
characteristics_dict = {0: ['disease state: control', 'disease state: SCZ', 'disease state: BD(-)', 'disease state: BD(+)'], 
                        1: ['laterality: left', 'laterality: right']}

# Create a correctly structured clinical data DataFrame
# This is in the format expected by geo_select_clinical_features
clinical_data = pd.DataFrame()
for key, values in characteristics_dict.items():
    # Create a Series and then transpose it to get a single row
    row_series = pd.Series(values)
    # Add as a row to the DataFrame
    clinical_data[key] = row_series

# 1. Gene Expression Data Availability
# Based on the title and summary, this appears to be gene expression data combined with DNA methylation
is_gene_available = True

# 2.1 Data Availability
# For trait (Bipolar_disorder)
# Looking at key 0, we can see "disease state: BD(+)" and "disease state: BD(-)" which indicate bipolar disorder cases
trait_row = 0

# Age data is not provided in the sample characteristics
age_row = None

# Gender data is not provided in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert disease state to binary trait (Bipolar_disorder)"""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # BD(+) and BD(-) both indicate Bipolar Disorder cases
    if value.startswith('BD'):
        return 1
    # Controls
    elif value.lower() == 'control':
        return 0
    # SCZ indicates Schizophrenia, not Bipolar
    elif value.lower() == 'scz':
        return 0
    else:
        return None

def convert_age(value):
    """Convert age to continuous values"""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# Define get_feature_data function needed by geo_select_clinical_features
def get_feature_data(df, row_idx, feature_name, convert_func):
    """Extract and process feature data from a row in the DataFrame."""
    values = df[row_idx].values
    processed_values = [convert_func(value) for value in values]
    return pd.DataFrame({feature_name: processed_values}, index=df.index)

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None

# Conduct initial filtering and save info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print(f"Preview of selected clinical features: {preview}")
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
        import traceback
        print(traceback.format_exc())


Preview of selected clinical features: {0: [0.0], 1: [nan]}
Clinical data saved to ../../output/preprocess/Bipolar_disorder/clinical_data/GSE120342.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Bipolar_disorder/GSE120342/GSE120342-GPL16311_series_matrix.txt.gz
Gene data shape: (19070, 30)
First 20 gene/probe identifiers:
Index(['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at',
       '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at',
       '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers provided, I can identify that these are DNA methylation probes,
# not gene symbols. The "cg" prefix is characteristic of Illumina DNA methylation arrays 
# (like the 450K or EPIC arrays). These need to be mapped to gene symbols if we want
# to associate them with specific genes.

# For methylation data, each probe corresponds to a specific CpG site in the genome,
# and these sites may be associated with specific genes, but they are not gene expression values
# in themselves, but rather represent DNA methylation levels.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check if there are any columns that might contain gene information
sample_row = gene_annotation.iloc[0].to_dict()
print("\nFirst row as dictionary:")
for col, value in sample_row.items():
    print(f"{col}: {value}")

# Check if IDs in gene_data match IDs in annotation
print("\nComparing gene data IDs with annotation IDs:")
print("First 5 gene data IDs:", gene_data.index[:5].tolist())
print("First 5 annotation IDs:", gene_annotation['ID'].head().tolist())

# Properly check for exact ID matches between gene data and annotation
gene_data_ids = set(gene_data.index)
annotation_ids = set(gene_annotation['ID'].astype(str))
matching_ids = gene_data_ids.intersection(annotation_ids)
id_match_percentage = len(matching_ids) / len(gene_data_ids) * 100 if len(gene_data_ids) > 0 else 0

print(f"\nExact ID match between gene data and annotation:")
print(f"Matching IDs: {len(matching_ids)} out of {len(gene_data_ids)} ({id_match_percentage:.2f}%)")

# Check which columns might contain gene symbols for mapping
potential_gene_symbol_cols = [col for col in gene_annotation.columns 
                             if any(term in col.upper() for term in ['GENE', 'SYMBOL', 'NAME'])]
print(f"\nPotential columns for gene symbols: {potential_gene_symbol_cols}")

# Check if the identified columns contain non-null values
for col in potential_gene_symbol_cols:
    non_null_count = gene_annotation[col].notnull().sum()
    non_null_percent = non_null_count / len(gene_annotation) * 100
    print(f"Column '{col}': {non_null_count} non-null values ({non_null_percent:.2f}%)")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Name', 'IlmnStrand', 'AddressA_ID', 'AlleleA_ProbeSeq', 'AddressB_ID', 'AlleleB_ProbeSeq', 'GenomeBuild', 'Chr', 'MapInfo', 'Ploidy', 'Species', 'Source', 'SourceVersion', 'SourceStrand', 'SourceSeq', 'TopGenomicSeq', 'Next_Base', 'Color_Channel', 'TSS_Coordinate', 'Gene_Strand', 'Gene_ID', 'Symbol', 'Synonym', 'Accession', 'GID', 'Annotation', 'Product', 'Distance_to_TSS', 'CPG_ISLAND', 'CPG_ISLAND_LOCATIONS', 'MIR_CPG_ISLAND', 'RANGE_GB', 'RANGE_START', 'RANGE_END', 'RANGE_STRAND', 'GB_ACC', 'ORF']
{'ID': ['cg00000292', 'cg00002426', 'cg00003994', 'cg00005847', 'cg00006414'], 'Name': ['cg00000292', 'cg00002426', 'cg00003994', 'cg00005847', 'cg00006414'], 'IlmnStrand': ['TOP', 'TOP', 'TOP', 'BOT', 'BOT'], 'AddressA_ID': [990370.0, 6580397.0, 7150184.0, 4850717.0, 6980731.0], 'AlleleA_ProbeSeq': ['AAACATTAATTACCAACCACTCTTCCAAAAAACACTTACCATTAAAACCA', 'AATATAATAACATTACCTTACCCATCTTATAATCAAACCAAACAAAAACA', 'AATAATAATAATACCCCCTAT

### Step 6: Gene Identifier Mapping

In [7]:
# Based on the previous outputs, I can identify:
# - Gene data IDs (index) are 'cg' prefixed probes
# - 'ID' column in gene_annotation matches these probe IDs
# - 'Symbol' column contains the gene symbols we want to map to

# 1. Decide the key mappings:
probe_id_col = 'ID'       # Column containing methylation probe IDs
gene_symbol_col = 'Symbol' # Column containing gene symbols

# 2. Extract mapping information using the get_gene_mapping function from the library
gene_mapping = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)
print(f"Gene mapping shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level data to gene expression data
# The function will handle the many-to-many mapping and distribution of expression values
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data after mapping: {gene_data.shape}")
print("First few genes and their expression values:")
print(gene_data.head())

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping shape: (27551, 2)
Gene mapping preview:
           ID    Gene
0  cg00000292  ATP2A1
1  cg00002426   SLMAP
2  cg00003994   MEOX2
3  cg00005847   HOXD3
4  cg00006414  ZNF398
Gene expression data after mapping: (0, 30)
First few genes and their expression values:
Empty DataFrame
Columns: [GSM3398477, GSM3398478, GSM3398479, GSM3398480, GSM3398481, GSM3398482, GSM3398483, GSM3398484, GSM3398485, GSM3398486, GSM3398487, GSM3398488, GSM3398489, GSM3398490, GSM3398491, GSM3398492, GSM3398493, GSM3398494, GSM3398495, GSM3398496, GSM3398497, GSM3398498, GSM3398499, GSM3398500, GSM3398501, GSM3398502, GSM3398503, GSM3398504, GSM3398505, GSM3398506]
Index: []

[0 rows x 30 columns]
Gene expression data saved to ../../output/preprocess/Bipolar_disorder/gene_data/GSE120342.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# First check the clinical data structure
print("Clinical data from previous steps:")
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(selected_clinical_df)

# Check sample compatibility
gene_samples = set(gene_data.columns)
clinical_indices = set(selected_clinical_df.index)
print(f"Gene data columns (samples): {list(gene_data.columns)[:5]}...")
print(f"Clinical data indices: {list(clinical_indices)}")

# Transpose clinical data to get it in the right format (features as rows)
clinical_df_t = selected_clinical_df.T
print("Transposed clinical data:")
print(clinical_df_t)

# Since the clinical data does not match the gene samples, we need to check the structure
# By checking the SOFT file content, we can see if there's better sample metadata
# Check if the sample identifiers in gene_data match GSM IDs
gsm_pattern = re.compile(r'GSM\d+')
gene_sample_matches = [bool(gsm_pattern.match(col)) for col in gene_data.columns]
print(f"Gene data columns match GSM pattern: {all(gene_sample_matches)}")

# Try to create a simple clinical DataFrame with trait data for all gene samples
if all(gene_sample_matches):
    # Extract the original BD status from sample characteristics
    bd_status = clinical_data.iloc[0].map(lambda x: 1 if isinstance(x, str) and 'BD' in x else 0)
    
    # Create a new clinical dataframe with gene samples
    new_clinical_df = pd.DataFrame({trait: 0}, index=gene_data.columns)
    # Set BD samples to 1
    for sample in gene_data.columns:
        if 'BD' in str(clinical_data.get(sample, '')):
            new_clinical_df.loc[sample, trait] = 1
    
    print("Created simple clinical dataframe:")
    print(new_clinical_df)
    
    # Link clinical and genetic data with the new clinical dataframe
    linked_data = geo_link_clinical_genetic_data(new_clinical_df.T, gene_data)
else:
    # Create a dummy clinical dataframe with all samples labeled as cases (1)
    # This is a fallback approach when metadata is insufficient
    print("Creating dummy clinical data for gene samples")
    dummy_clinical_df = pd.DataFrame({trait: 1}, index=gene_data.columns)
    linked_data = geo_link_clinical_genetic_data(dummy_clinical_df.T, gene_data)

print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
try:
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
except Exception as e:
    print(f"Error checking for bias: {e}")
    is_biased = True  # Assume biased if there's an error

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="SuperSeries with DNA methylation data mapped to genes. Clinical annotations are limited."
)

# 6. Save the linked data if usable
if is_usable and not linked_data.empty:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene data shape after normalization: (0, 30)
Normalized gene expression data saved to ../../output/preprocess/Bipolar_disorder/gene_data/GSE120342.csv
Clinical data from previous steps:
Selected clinical data shape: (1, 2)
Clinical data preview:
                    0   1
Bipolar_disorder  0.0 NaN
Gene data columns (samples): ['GSM3398477', 'GSM3398478', 'GSM3398479', 'GSM3398480', 'GSM3398481']...
Clinical data indices: ['Bipolar_disorder']
Transposed clinical data:
   Bipolar_disorder
0               0.0
1               NaN
Gene data columns match GSM pattern: True
Created simple clinical dataframe:
            Bipolar_disorder
GSM3398477                 0
GSM3398478                 0
GSM3398479                 0
GSM3398480                 0
GSM3398481                 0
GSM3398482                 0
GSM3398483                 0
GSM3398484                 0
GSM3398485                 0
GSM3398486                 0
GSM3398487                 0
GSM3398488                 0
GSM3398489     